In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


In [2]:
import geopandas
from geopandas import GeoDataFrame
from shapely.geometry import Point
import pyproj
from pyproj import Proj, transform

In [3]:
df = pd.read_csv('../data/LADOT_Metered_Parking_Inventory___Policies.csv')

In [4]:
df.head(1)

,SpaceID,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng
0,VS336,4900 SUNSET BL,Single-Space,FLAT,$1.00,2HR,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",NaN,"(34.098024, -118.296664)"


In [5]:
df.shape

(33989, 9)

In [6]:
df['Latitude'] = df['LatLng'].str.replace("(", "").str.split().str[0]
df['Longitude'] = df['LatLng'].str.replace(")","").str.split().str[1]
df["Latitude"] = df['Latitude'].str[:-1].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [7]:
import gmaps


gmaps.configure(api_key='') # Fill in with your API key



locations = df[["Latitude","Longitude"]]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations))
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
mapbox_access_token = 'pk.eyJ1IjoiZWxseWt5bGVzIiwiYSI6ImNrYWZvZ2dmdjAwaW0ycm81bGg3Yjh2c2wifQ.QN0HHPXMY1S1bDClQ1eYEA'
import json
import geopandas 
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon

In [35]:
#"Latitude","Longitude"
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Creating a Geographic data frame 
gdf = geopandas.GeoDataFrame(df, crs=crs, geometry=geometry)

In [36]:
gdf.head(1)

,SpaceID,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng,Latitude,Longitude,geometry
0,VS336,4900 SUNSET BL,Single-Space,FLAT,$1.00,2HR,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",NaN,"(34.098024, -118.296664)",34.098024,-118.296664,POINT (-118.29666 34.09802)


In [37]:
gdf_group=gdf.groupby(['BlockFace', 'MeterType','RateType','RateRange','MeteredTimeLimit','ParkingPolicy']).agg({'LatLng': lambda x : ', '.join(x)})


In [38]:
gdf_group=gdf_group.reset_index()

In [39]:
# Make the list of the coordinates
gdf_group['LatLng'] = gdf_group['LatLng'].str.split(',')
gdf_group.head(5)

,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,LatLng
0,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,"[(33.984622, -118.470849), (33.98493, -118...."
1,1 WASHINGTON BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,"[(33.979206, -118.466809)]"
2,10 E MARKET ST,Multi-Space,SEASONAL,$1 - FW/$2 - SS,1HR,8A-6P Daily,"[(33.988009, -118.473198), (33.988057, -118..."
3,10 E WINDWARD AVE,Multi-Space,SEASONAL,$1 - FW/$2 - SS,1HR,8A-6P Daily,"[(33.987405, -118.472604), (33.987257, -118..."
4,10 W WASHINGTON BLVD,Multi-Space,SEASONAL,$1 - FW/$2 - SS,2HR,8A-6P Daily,"[(33.979534, -118.465848), (33.979316, -118..."


In [40]:
from shapely import geometry

poly = geometry.Polygon([[p.x, p.y] for p in pointList])

NameError: name 'pointList' is not defined

In [41]:
blockface = gdf['BlockFace'].value_counts()
blockface

5900 6TH ST                  46
3101 GLENDALE BL             44
10900 W SANTA MONICA BLVD    41
601 LA FAYETTE PK PL         40
600 SAN VICENTE BL           40
                             ..
2701 ROWENA AV                1
6700 HOLLYWOOD BL             1
4601 OAKWOOD AV               1
700 ORLANDO AV                1
5700 2ND ST                   1
Name: BlockFace, Length: 4675, dtype: int64

In [42]:
block_index_ploy = {}
for block_index in blockface.index:
    geo_list = []
    for geo in gdf[gdf['BlockFace'] == block_index]['geometry']:
        geo_list.append(geo)
    block_index_ploy[block_index] = geo_list
    

In [43]:
block_pd = pd.DataFrame(block_index_ploy.items(), index = block_index_ploy.keys())

In [44]:
block_pd.head()

,0,1
5900 6TH ST,5900 6TH ST,"[POINT (-118.356895 34.064787), POINT (-118.35..."
3101 GLENDALE BL,3101 GLENDALE BL,"[POINT (-118.262405 34.116822), POINT (-118.26..."
10900 W SANTA MONICA BLVD,10900 W SANTA MONICA BLVD,"[POINT (-118.43847 34.049129), POINT (-118.439..."
601 LA FAYETTE PK PL,601 LA FAYETTE PK PL,"[POINT (-118.282991 34.061985), POINT (-118.28..."
600 SAN VICENTE BL,600 SAN VICENTE BL,"[POINT (-118.371769 34.064534), POINT (-118.37..."


In [45]:
from_json = gdf.to_json()
geoJSON = json.loads(from_json)

In [46]:
geoJSON['features'][:2]

[{'id': '0',
  'type': 'Feature',
  'properties': {'BlockFace': '4900 SUNSET BL',
   'LatLng': '(34.098024, -118.296664)',
   'Latitude': 34.098024,
   'Longitude': -118.296664,
   'MeterType': 'Single-Space',
   'MeteredTimeLimit': '2HR',
   'ParkingPolicy': 'TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat',
   'RateRange': '$1.00',
   'RateType': 'FLAT',
   'SpaceID': 'VS336',
   'StreetCleaning': None},
  'geometry': {'type': 'Point', 'coordinates': [-118.296664, 34.098024]}},
 {'id': '1',
  'type': 'Feature',
  'properties': {'BlockFace': '700 E PICO BLVD',
   'LatLng': '(34.032054, -118.252521)',
   'Latitude': 34.032054,
   'Longitude': -118.252521,
   'MeterType': 'Single-Space',
   'MeteredTimeLimit': '2HR',
   'ParkingPolicy': '8A-6P Mon-Sat',
   'RateRange': '$0.5 - $1',
   'RateType': 'TOD',
   'SpaceID': 'ED396',
   'StreetCleaning': None},
  'geometry': {'type': 'Point', 'coordinates': [-118.252521, 34.032054]}}]

In [26]:
with open('geojson','w') as f:
    json.dump(geoJSON,f)

In [27]:
pts=[]#list of points defining boundaries of polygons
for  feature in geoJSON['features']:
    if feature['geometry']['type']=='Point':
        pts.extend(feature['geometry']['coordinates'])    
        #pts.append([None, None])#mark the end of a polygon   

    elif feature['geometry']['type']=='MultiPolygon':
        for polyg in feature['geometry']['coordinates']:
            pts.extend(polyg[0])
            pts.append([None, None])#end of polygon
    else: raise ValueError("geometry type irrelevant for map") 

In [28]:
X = []
Y = []
for i in range(len(pts)):        
    if i % 2 == 0:
        X.append(pts[i])
    else:
        Y.append(pts[i])
        

In [29]:
X[:10]

[-118.296664,
 -118.252521,
 -118.264675,
 -118.274463,
 -118.44891,
 -118.344672,
 -118.27393,
 -118.266015,
 -118.369344,
 -118.445724]

In [51]:
data = [
go.Scattermapbox(
    lat=Y,
    lon=X,
    mode='markers',
    marker=dict(size= 3,color = 'red',opacity = .8))
    ]

layout = go.Layout(autosize=True,
                   mapbox = dict(center= dict(lat=33,     #Set center at Flataklocken
                                 lon=-118),        
                                 accesstoken=mapbox_access_token,
                                 zoom=4,
                                 style='light',
                               ),
                   
                    width=1000,
                    height=1000,
                    title = 'LA')


fig = go.Figure(data=data, layout=layout)
plot(fig, filename='LA Mapbox.html')

'LA Mapbox.html'

In [53]:
from IPython.display import IFrame
IFrame(src='LA Mapbox.html', width=700, height = 700)